In [62]:
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd

In [2]:
import os
from pyspark import SparkFiles
import zipfile
import io
from config import FTP_DATASOURCE, FTP_DATASOURCE_URI

In [3]:
# inicialize session and get context spark
spark = SparkSession.builder.appName("teste").getOrCreate()
sc = spark.sparkContext

In [31]:
#function to extract and save database path
def zip_extract_save(x):
    in_memory_data = io.BytesIO(x[1])
    with zipfile.ZipFile(in_memory_data, "r") as zipped:
            zipped.extractall("database")
    
zips = sc.binaryFiles(FTP_DATASOURCE_URI)
files_data = zips.map(zip_extract_save)
files_data.collect()

In [93]:
#function unzip save dataframe pandas
def zip_extract_dataframe_panda(x):
    in_memory_data = io.BytesIO(x[1])
    file_obj = zipfile.ZipFile(in_memory_data, "r")
    #files = [i for i in file_obj.namelist()]
    for file in file_obj.namelist():
        f = file_obj.open(file)
        df = pd.read_csv(f, sep=';')
        #dict(zip(files, [file_obj.open(file).read() for file in files]))        
    return df


zips = sc.binaryFiles(FTP_DATASOURCE_URI)
files_data = zips.map(zip_extract)
files_data.collect()

[  TRANSACAO       MUNICIPIO ESTADO DATA_ATUALIZACAO  Unnamed: 4
 0        10      UBERLANDIA     MG       23/02/2021         NaN
 1        50       SAO PAULO     SP       24/02/2021         NaN
 2         5       SAO PAULO     SP       25/02/2021         NaN
 3        20      UBERLANDIA     MG       23/02/2021         NaN
 4       100  RIO DE JANEIRO     RJ       23/02/2021         NaN
 5     200,2  RIO DE JANEIRO     RJ       24/02/2021         NaN
 6     50,41        CAMPINAS     SP       24/02/2021         NaN
 7        30        CAMPINAS     SP       24/02/2021         NaN
 8       500       SAO PAULO     SP       25/02/2021         NaN]

In [ ]:
#Lendo arquivo que foi salvo e reescrevendo

In [56]:
df = spark.read.csv('database/data_municipio.csv', sep=';',header=True, ignoreTrailingWhiteSpace=True)
df = df.drop('_c4')

In [82]:
# savando como parquet0
df.repartition(8).write.format('parquet').save('database/data_municipio.parquet')